In [1]:
import json
import logging

import boto3
from botocore.exceptions import ClientError

logger = logging.getLogger(__name__)

In [8]:
logger.setLevel("DEBUG")

In [9]:
# Test single prompt inference, to make sure connections work
def invoke_claude_3_with_text(
    prompt: str, model_id: str = "anthropic.claude-3-sonnet-20240229-v1:0"
) -> dict:
    """
    Invokes Anthropic Claude 3 Sonnet to run an inference using the input
    provided in the request body.

    :param prompt: The prompt that you want Claude 3 to complete.
    :param model_id: The bedrock model ID to use for summarization
    :return: Inference response from the model.
    """

    # Initialize the Amazon Bedrock runtime client
    client = boto3.client(service_name="bedrock-runtime", region_name="us-east-1")

    try:
        response = client.invoke_model(
            modelId=model_id,
            body=json.dumps(
                {
                    "anthropic_version": "bedrock-2023-05-31",
                    "max_tokens": 5000,
                    "messages": [
                        {
                            "role": "user",
                            "content": [{"type": "text", "text": prompt}],
                        }
                    ],
                    "temperature": 1,
                    "top_p": 0.999,
                    "top_k": 250,
                }
            ),
        )

        # Process and print the response
        result = json.loads(response.get("body").read())
        input_tokens = result["usage"]["input_tokens"]
        output_tokens = result["usage"]["output_tokens"]
        output_list = result.get("content", [])

        logger.debug("Invocation details:")
        logger.debug(f"- The input length is {input_tokens} tokens.")
        logger.debug(f"- The output length is {output_tokens} tokens.")

        logger.debug(f"- The model returned {len(output_list)} response(s):")
        for output in output_list:
            logger.debug(output["text"])

        return result

    except ClientError as err:
        logger.error(
            "Couldn't invoke Claude 3 Sonnet. Here's why: %s: %s",
            err.response["Error"]["Code"],
            err.response["Error"]["Message"],
        )
        raise

In [10]:
def gen_summary_from_transcript(
    transcript: str, model_id: str = "anthropic.claude-3-sonnet-20240229-v1:0"
) -> str:
    FULL_PROMPT = (
        "I am a consultant building a generative AI application for a customer. "
        "We are in the early phases of the engagement, and have conducted a "
        "discovery workshop. The purpose of the workshop was to concretely "
        "determine the scope of the POC, responsibilities, success metrics, "
        "and next steps. I transcribed a recording of the workshop and will "
        "provide the transcript to you below. Please write me a one page summary "
        "of the meeting, emphasizing things like use case description, next steps, "
        "potential obstacles identified, success criteria, and timelines. Here is "
        "the meeting transcript:\n"
        "<transcript>\n"
        f"{transcript}\n"
        "</transcript>"
    )

    res = invoke_claude_3_with_text(FULL_PROMPT)
    output_list = res.get("content", [])
    assert len(output_list) == 1, f"Output list has len {len(output_list)}, expected 1."
    return output_list[0]["text"]

In [11]:
fake_transcript = "hello thanks for attending the meeting about AMERICAN FOOTBALL!"
kazu = gen_summary_from_transcript(fake_transcript)

In [7]:
kazu

'Unfortunately, the provided transcript does not contain any relevant information about a generative AI application or a discovery workshop. The transcript only says "hello thanks for attending the meeting about AMERICAN FOOTBALL!".\n\nWithout any actual details from the discovery workshop discussion, I cannot provide a meaningful one-page summary covering aspects such as the use case description, next steps, potential obstacles, success criteria, and timelines for the generative AI application project.\n\nPlease provide the full and accurate transcript of the discovery workshop to allow me to summarize the key points and outcomes effectively.'